In [2]:
import os
import requests
import bs4
from bs4 import BeautifulSoup
import sys
import pandas as pd
import time
import matplotlib.pyplot as plt
import seaborn as sb
import numpy as np

In [3]:
print(bs4.__version__)

4.9.3


In [8]:
url = 'https://www.tripadvisor.com.br/Restaurants-g303631-Sao_Paulo_State_of_Sao_Paulo.html'
headers = ({'User-Agent':
            'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36'})
resposta=requests.get(url=url,headers=headers)

resposta.encoding='utf-8'

In [10]:
print('Código de status da resposta: {}'.format(resposta.status_code))

Código de status da resposta: 200


In [26]:
soup=BeautifulSoup(resposta.text,'html.parser')

In [30]:
tag_restaurantes=soup.find('div',class_="_1kXteagE")

In [44]:
lista_restaurantes=tag_restaurantes.find_all('div',class_="_1llCuDZj")

In [46]:
lista_restaurantes[0]

<div class="_1llCuDZj" data-test="SL_list_item"><span><div><div class="_1kNOY9zw"><div class="_2jF2URLh"><span><a class="_2uEVo25r _3tdrXOp7" href="/Restaurant_Review-g303631-d6557384-Reviews-Bacalhau_Vinho_e_Cia_Barra_Funda-Sao_Paulo_State_of_Sao_Paulo.html" target="_blank"><div class="_2PhdrMP3"></div><div class="_2KPNYP9B" data-clicksource="Photo"></div></a></span><div><div class="_2QJ7e_cW"><div class="_15aaPtHM _50cmFIkv D1m_VFgO _2a-PzfE-"><span class="_1YkiZl2_ _2geKhlYH"><span class="_1tvInqmQ _1T_s6QZp"><svg class="_3nS1tofR iG08Yf8B" height="1em" viewbox="0 0 24 24" width="1em"><path d="M12 2C6.487 2 2 6.487 2 12c0 5.515 4.487 10 10 10 5.515 0 10-4.485 10-10 0-5.513-4.485-10-10-10zm4.688 10.911c-.975 1.188-4.687 4.434-4.687 4.434S8.258 14.1 7.29 12.903c-1.14-1.411-1.12-3.241.049-4.351.611-.58 1.42-.898 2.279-.898s1.668.318 2.279.898l.096.092.09-.087a3.296 3.296 0 012.278-.897c.86 0 1.669.318 2.277.897 1.201 1.139 1.219 2.929.05 4.354z"></path></svg></span><span class="_1tvInq

Para cada restaurante:

    -Nome 
    
    -"Estrelas"(0 a 5)
    
    -Avaliações
    
    -Cozinha (steakhouse,italiano,frutos do mar,...)
    
    -Preço 
    
    -Há medidas de segurança? (div class="_1ULcQ4O5 _2TXtEj3w")
    
    -Review
    

In [ ]:
for i in range (len(lista_restaurantes)):
    